In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install datasets transformers accelerate huggingface-hub torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 525.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.5.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from datasets import load_dataset
from huggingface_hub import notebook_login
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline, AutoModelForSeq2SeqLM

In [9]:
notebook_login()

In [14]:
model = 'Josmar/BART_Finetuned_CNN_dailymail'
dataset_name = 'abisee/cnn_dailymail'
version = '3.0.0'

In [11]:
dataset = load_dataset(dataset_name, version)
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSeq2SeqLM.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [16]:
def format_data(data_point):
  input_encodings = tokenizer(data_point['article'], truncation=True, padding='max_length', max_length=1024)
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(data_point['highlights'], truncation=True, padding='max_length', max_length=128)

  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

cnndaily_modified = dataset.map(format_data, batched=True)
cnndaily_modified

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [18]:
# def format_data(data_point):
#   prompt = f"""
#   ### instruction : {"generate the summary for the given piece of corpus"}
#   ### input : {data_point['article']}
#   ### output :{data_point['highlights']}
#   """
#   tokens = tokenizer(
#     prompt,
#     truncation = True,
#     max_length = 256,
#     padding = "max_length"
#   )
#   tokens['labels'] = tokens['input_ids'].copy()
#   return tokens

training_data = cnndaily_modified['train'].take(500)
eval_data = cnndaily_modified['validation'].take(100)
testing_data = cnndaily_modified['test'].take(100)

In [19]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs = 2,
    warmup_steps=5,
    learning_rate=5e-05,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    weight_decay=0.01,
    logging_steps=10,
    lr_scheduler_type='linear',
    eval_strategy='steps',
    eval_steps=3,

)


trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=training_data,
    eval_dataset=eval_data
)

trainer.train()

Step,Training Loss,Validation Loss
3,No log,9.746529
6,No log,8.446907
9,No log,7.354518
12,8.352000,5.766981
15,8.352000,4.082030
18,8.352000,3.307424
21,4.740500,2.763605
24,4.740500,2.401784
27,4.740500,2.130604
30,2.822200,1.910024


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


TrainOutput(global_step=84, training_loss=2.8807797999609086, metrics={'train_runtime': 301.092, 'train_samples_per_second': 3.321, 'train_steps_per_second': 0.279, 'total_flos': 609736458240000.0, 'train_loss': 2.8807797999609086, 'epoch': 2.0})

In [21]:
trainer.save_model('bart-finetuned')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


In [25]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = cnndaily_modified["test"][2]["article"]

highlight  = cnndaily_modified["test"][2]["highlights"]

pipe = pipeline("summarization", model="bart-finetuned")

# model_x = AutoModelForCausalLM.from_pretrained('bart-finetuned')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [23]:
# input = testing_data[0]
trainer.evaluate(testing_data)


{'eval_loss': 0.9219788908958435,
 'eval_runtime': 4.6322,
 'eval_samples_per_second': 21.588,
 'eval_steps_per_second': 1.943,
 'epoch': 2.0}

In [27]:
 ##
print("article:")
print(sample_text)


print("\nReference Summary:")
print(highlight)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

article:
(CNN)If you've been following the news lately, there are certain things you doubtless know about Mohammad Javad Zarif. He is, of course, the Iranian foreign minister. He has been U.S. Secretary of State John Kerry's opposite number in securing a breakthrough in nuclear discussions that could lead to an end to sanctions against Iran -- if the details can be worked out in the coming weeks. And he received a hero's welcome as he arrived in Iran on a sunny Friday morning. "Long live Zarif," crowds chanted as his car rolled slowly down the packed street. You may well have read that he is "polished" and, unusually for one burdened with such weighty issues, "jovial." An Internet search for "Mohammad Javad Zarif" and "jovial" yields thousands of results. He certainly has gone a long way to bring Iran in from the cold and allow it to rejoin the international community. But there are some facts about Zarif that are less well-known. Here are six: . In September 2013, Zarif tweeted "Happy